In [11]:
# Imports

# Data handling
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import ast


In [13]:
# Read POIS files
BOTA = pd.read_csv('your_path/LABOTA_POI.csv', sep='|')
BOTA['zona_estudio'] = 'LA BOTA'
TOCTIUCO = pd.read_csv('your_path/TOCTIUCO_POI.csv', sep='|')
TOCTIUCO['zona_estudio'] = 'TOCTIUCO'

# Make a single table
POIS = pd.concat([BOTA,TOCTIUCO],axis=0)
POIS = POIS[POIS['business_status'] == 'OPERATIONAL']
POIS = POIS[['geometry','lat','lon','name','rating','types', 'zona_estudio']]

In [14]:
# Change list to string
POIS['types'] = POIS['types'].apply(ast.literal_eval)
# Remove unwanted
remove_unwanted = lambda l: [x for x in l if x not in ['point_of_interest', 'establishment']]
# Format types
POIS['types'] = POIS['types'].apply(remove_unwanted)
POIS['n_types'] = POIS['types'].apply(lambda x: len(x))
max_list_len = max(POIS['n_types'])
type_cols = [f'type_{i}' for i in range(max_list_len)]
POIS[type_cols] = POIS['types'].apply(pd.Series)
POIS = POIS.fillna(np.nan)
# Category join
categories = pd.read_csv('your_path/POI_categories.csv', sep=',')
POIS = POIS[['geometry','lat','lon','name','rating','type_0', 'zona_estudio']]
POIS = POIS.rename(columns={'type_0': 'type'})
POIS = POIS.merge(categories, on='type', how='left')

<ipython-input-14-c066845378b3>:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  POIS[type_cols] = POIS['types'].apply(pd.Series)


In [15]:
POIS.head(10)

,geometry,lat,lon,name,rating,type,zona_estudio,category,sub_cat
0,POINT (-78.4573401 -0.1099149),-0.109915,-78.457340,Mayflower,4.0,restaurant,LA BOTA,local_economy,food and drinks
1,POINT (-78.4652594 -0.1198871),-0.119887,-78.465259,Los Encebollados de Panchito,4.5,restaurant,LA BOTA,local_economy,food and drinks
2,POINT (-78.4575256 -0.1084616),-0.108462,-78.457526,Ch Farina,4.2,restaurant,LA BOTA,local_economy,food and drinks
3,POINT (-78.46539229999999 -0.1229637),-0.122964,-78.465392,La casucha del manaba,4.5,restaurant,LA BOTA,local_economy,food and drinks
4,POINT (-78.4533703 -0.1075254),-0.107525,-78.453370,Pizza Hut Carapungo,4.2,meal_delivery,LA BOTA,local_economy,food and drinks
5,POINT (-78.452833 -0.1078888),-0.107889,-78.452833,Los Ceviches de la Rumiñahui,4.1,restaurant,LA BOTA,local_economy,food and drinks
6,POINT (-78.4535535 -0.1061874),-0.106187,-78.453553,La Ramada Manabita,4.6,restaurant,LA BOTA,local_economy,food and drinks
7,POINT (-78.467528 -0.1241037),-0.124104,-78.467528,Cai Lin,4.3,restaurant,LA BOTA,local_economy,food and drinks
8,POINT (-78.453364 -0.104081),-0.104081,-78.453364,KFC,4.0,meal_takeaway,LA BOTA,local_economy,food and drinks
9,POINT (-78.4598559 -0.1156608),-0.115661,-78.459856,POLLO HORNEADO MY CHICKEN,4.0,restaurant,LA BOTA,local_economy,food and drinks


In [17]:
POIS.to_csv('your_path/POIS_final.csv', index=False, sep = '|')